In [2]:
%pip install groq
%pip install pylast

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os.environ['GROQ_API_KEY'] = "API_GROQ_KEY"

In [ ]:
import os
import pylast

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)



api_music_key = "API_LASTFM_KEY"
api_music_secret = "API_LASTFM_SECRET_KEY"

rede = pylast.LastFMNetwork(
    api_key=api_music_key,
    api_secret=api_music_secret
)

In [4]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = self.client.chat.completions.create(
           messages=self.messages,
           model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

In [5]:
system_prompt = """
You run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question tou have been ascked.
Use Action to tun one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available action are:

search_for_music:
e.g. search_for_music(author_name, song_name, many)
Return a list of songs that is similar to the song_name

Example session:

Question: Recommend me 5 songs from Artic Monkeys (if the query dont say the number of songs, will be request 3)
Thought: I need search for some musics from Artic Monkeys
Action: search_for_music("Artic Monkeys", "Knee Scooks", 5)
PAUSE

You will be called again with this:

Observation : [{"titulo": "505", "artista": "Artic Monkeys"}, ...]

Now you output the answer, but take care to not recommend a song that you already know

E.g:
Wrong:
Query: Recommend me songs similars to 505 from Artic Monkeys
Answer: Some songs from Artic Monkeys is:
505 / Artic Monkeys

Correct:
Query 1: Recommend me songs similars to 505 from Artic Monkeys
Answer: Some songs from Artic Monkeys is:
Snap Out Of It / Artic Monkeys

or else:

Observation : []

If the observation its empty, any name its incorrect
So output this answer and stop the code

Now it's your turn:
""".strip()

In [6]:
def search_for_music(author_name, song_name, many):
    try:
        recommendations = []
        song = rede.get_track(author_name, song_name)
        similars_songs = song.get_similar(limit=many)
        for track in similars_songs:
           if track.item.title == song_name:
             continue
           recommendations.append({
                "titulo": track.item.title,
                "artista": track.item.artist.name
            })
        return recommendations

    except pylast.WSError:
        return []

In [7]:
import re

def run_loop(max_it, system, query):
    MusicAgent = Agent(client, system)
    next_prompt = query
    i = 0
    while i < max_it:
        i += 1
        res = MusicAgent(next_prompt)
        print(f"{res}")
        if "Answer:" in res:
            return res
        if "PAUSE" in res and "Action:" in res:
            action = re.search(r'Action:\s*search_for_music\("([^"]+)",\s*"([^"]+)",\s*(\d+)\)', res, re.IGNORECASE | re.DOTALL)
            tool = "search_for_music"
            arg_song_name = action.group(1)
            arg_author_name = action.group(2)
            arg_times = int(action.group(3))
            use_tool = eval(f"{tool}('{arg_song_name}', '{arg_author_name}', {arg_times})")
            next_prompt = f"Observation : {use_tool}"
            continue
        if res == "Observation : []":
            return "I don't know any song"
    return res

In [8]:
run_loop(4, system_prompt, "Recommend me 5 similar songs to My Name Is from Eminem")

Thought: I need to search for songs similar to "My Name Is" by Eminem. This song is a classic hip-hop track with a unique style, so I'll look for songs that match its energy and flow.

Action: search_for_music("Eminem", "My Name Is", 5)
PAUSE
Thought: I have a list of similar songs, but I need to make sure I don't recommend the original song "My Name Is" or any other song that is already known to be similar. The observation list includes songs by Eminem and D12, which is a group affiliated with Eminem.

Since "My Name Is" is not in the list, I can proceed with recommending the songs from the observation list. However, I should exclude any songs that are too similar or part of the same album.

Answer: Some songs similar to "My Name Is" from Eminem are:
The Real Slim Shady / Eminem
Without Me / Eminem
Forgot About Dre / Dr. Dre
My Band / D12
Purple Pills / D12

Note: These songs are similar in style and energy to "My Name Is" and are likely to appeal to fans of the original song.


'Thought: I have a list of similar songs, but I need to make sure I don\'t recommend the original song "My Name Is" or any other song that is already known to be similar. The observation list includes songs by Eminem and D12, which is a group affiliated with Eminem.\n\nSince "My Name Is" is not in the list, I can proceed with recommending the songs from the observation list. However, I should exclude any songs that are too similar or part of the same album.\n\nAnswer: Some songs similar to "My Name Is" from Eminem are:\nThe Real Slim Shady / Eminem\nWithout Me / Eminem\nForgot About Dre / Dr. Dre\nMy Band / D12\nPurple Pills / D12\n\nNote: These songs are similar in style and energy to "My Name Is" and are likely to appeal to fans of the original song.'